# Authentication

The `auth` API uses cryptographic authenticated to verify a message has not been altered with a tag and secret key that all parties must share.

[Libsodium Documentation](https://doc.libsodium.org/secret-key_cryptography/secret-key_authentication)

Functions:
```
    crypto_auth_keygen() -> bytea

    crypto_auth(message bytea, key bytea) -> bytea

    crypto_auth(message bytea, key_id bigint, context bytea = 'pgsodium') -> bytea

    crypto_auth_verify(mac bytea, message bytea, key bytea) -> boolean

    crypto_auth_verify(mac bytea, message bytea, key_id bigint, context bytea = 'pgsodium') -> boolean
```

`crypto_auth_keygen()` generates a message-signing key for use by
`crypto_auth()`.  The role `pgsodium_keymaker` is required to call this function.

`crypto_auth()` generates an authentication tag (mac) for a
combination of message and secret key.  This does not encrypt the
message; it simply provides a means to prove that the message has not
been tampered with.  To verify a message tagged in this way, use
`crypto_auth_verify()`.  This function is deterministic: for a given
message and key, the generated mac will always be the same.  The role `pgsodium_keyholder` is required to call the raw `key bytea` versions of these functions.  The key id versions of the functions can be called with the role `pgsodium_keyiduser`.

Note that this requires access to the secret
key, which is not something that should normally be shared.  If
many users need to verify message it is usually better to use
[Public Key Signatures](#user-content-public-key-signatures) rather
than sharing secret keys.


In [1]:
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.displaycon=False
%sql postgresql://postgres@/

Encryption requires a key and a nonce.  The nonce doesn't have to be confidential, but it should never ever be reused with the same key. The easiest way to generate a nonce is to use `crypto_secretbox_noncegen`:

In [2]:
key = %sql select pgsodium.crypto_auth_keygen()::text
key = key[0][0]

## Signing

A new signature is created with the message and the key:

In [3]:
signature = %sql SELECT crypto_auth::text from pgsodium.crypto_auth('bob is your uncle', (:key)::bytea)
signature = signature[0][0]
print('The signature is: ', signature)

The signature is:  \x048a21486e401e4899ec6229cda326af564c41cd1b9baad57a41c3e78aaed752


## Verification

Using the tag and key, verify the authenticity of the message.

In [4]:
%sql SELECT crypto_auth_verify FROM pgsodium.crypto_auth_verify(:signature, 'bob is your uncle', (:key)::bytea)


crypto_auth_verify
True
